HEURISTIC 1

In [65]:
import numpy as np
import random
import pandas as pd
import copy
from collections import defaultdict
from networkx import nx



In [88]:
adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n100-m400.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2


In [89]:
#parameters
Blocking_Capacity = (int(n/10))*2
step_size=3

In [90]:
#sets

setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()


In [91]:
#Preprocessing
preprocessed_matrix = adjacency_df.copy(deep=True)



In [92]:
#2 infected 
preprocessed_matrix.loc[setP,setP] = 0




In [93]:
#Sum of all edges lower than threshold (not infected)
index = preprocessed_matrix.loc[(preprocessed_matrix[setV].sum(axis=1)<((preprocessed_matrix.loc[:, "Threshold"]))) & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)  , range(1,n+1)].index
preprocessed_matrix.loc[index,setV] = 0
preprocessed_matrix.loc[setV,index] = 0



In [94]:
#1 neighbor (not infected not risky)
index2 = preprocessed_matrix.loc[((preprocessed_matrix.loc[setV,setV]>0).sum(axis=1) == 1 & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)) & (preprocessed_matrix.loc[:,"IsRisky"]<0.01), range(1,n+1)].index
preprocessed_matrix.loc[index2,setV] = 0
preprocessed_matrix.loc[setV,index2] = 0

In [95]:
#Percentage assignment
preprocessed_matrix.loc[:,setV]=preprocessed_matrix.loc[:,setV].div(preprocessed_matrix.loc[:,"Threshold"],axis=0)
preprocessed_matrix[preprocessed_matrix[setV] >= 1] = 1


In [96]:
#!!!!!!!!! Edges below %50 influence are deleted !!!!!!!!!!!
preprocessed_matrix[preprocessed_matrix[setV] <= 0.3] = 0

In [97]:
def convert(a):      
    rows, cols = np.where(preprocessed_matrix.loc[range(1,n+1), range(1,n+1)] > 0)
    edges = zip((rows+1).tolist(), (cols+1).tolist())
    G = nx.Graph()
    G.add_edges_from(edges)

    return G

In [98]:
Graph = convert(preprocessed_matrix)
#Graph.nodes()

In [99]:
found_paths=[]
for starting_nodes in setP:
    for ending_nodes in setRisky:
        found_paths=found_paths+list(nx.all_simple_paths(Graph, starting_nodes, ending_nodes, cutoff=step_size))  

In [100]:
found_paths

[[16, 43, 51, 10],
 [16, 82, 51, 10],
 [16, 82, 71, 10],
 [16, 65, 58, 11],
 [16, 43, 7, 11],
 [16, 43, 20, 11],
 [16, 35, 94, 13],
 [16, 65, 13],
 [16, 65, 81, 13],
 [16, 43, 65, 13],
 [16, 82, 81, 13],
 [16, 99, 46, 15],
 [16, 43, 32, 15],
 [16, 65, 75, 24],
 [16, 73, 27, 24],
 [16, 43, 20, 24],
 [16, 43, 61, 24],
 [16, 73, 25],
 [16, 43, 73, 25],
 [16, 35, 26],
 [16, 35, 94, 26],
 [16, 99, 46, 26],
 [16, 43, 18, 26],
 [16, 82, 37, 26],
 [16, 65, 74, 30],
 [16, 99, 30],
 [16, 43, 32, 30],
 [16, 65, 43, 32],
 [16, 73, 27, 32],
 [16, 73, 43, 32],
 [16, 99, 30, 32],
 [16, 43, 32],
 [16, 35],
 [16, 82, 87, 35],
 [16, 35, 94, 36],
 [16, 43, 32, 36],
 [16, 35, 26, 37],
 [16, 65, 45, 37],
 [16, 65, 81, 37],
 [16, 73, 45, 37],
 [16, 43, 20, 37],
 [16, 82, 37],
 [16, 82, 81, 37],
 [16, 35, 87, 40],
 [16, 73, 40],
 [16, 43, 73, 40],
 [16, 82, 87, 40],
 [16, 65, 45],
 [16, 65, 58, 45],
 [16, 65, 75, 45],
 [16, 73, 45],
 [16, 43, 61, 45],
 [16, 43, 65, 45],
 [16, 43, 73, 45],
 [16, 82, 37, 45],


In [101]:
for j in setP:
    found_paths = [[ele for ele in sub if ele != j] for sub in found_paths]

In [102]:
remaining_blocking_capacity=Blocking_Capacity
found_paths_removable=found_paths
already_blocked_nodes=[]
found_paths_np = np.concatenate([np.array(i) for i in found_paths])

In [103]:
nodes_on_paths, total_appearance = np.unique(found_paths_np, return_counts=True)

for i in setP:
    total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))
#for i in setRisky:
    #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))    


In [104]:
#nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]



In [105]:
already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
remaining_blocking_capacity -= 1

In [106]:
while (remaining_blocking_capacity>0):
    
    
    found_paths_removable = list(filter(lambda x: already_blocked_nodes[-1] not in x, found_paths_removable))
    
    if (len(found_paths_removable)<=0):
        remaining_risky=np.array(setRisky)

        for i in already_blocked_nodes:
            remaining_risky=np.delete(remaining_risky, np.where(remaining_risky[:] == i))

        if(len(remaining_risky)>0):
            if(len(remaining_risky)>remaining_blocking_capacity):
                chosen=(np.random.choice(remaining_risky,size=remaining_blocking_capacity,replace=False))  
                for items in chosen:
                    already_blocked_nodes.append(items)
            else:
                 for items in remaining_risky:
                    already_blocked_nodes.append(items)   
        
        break
    
    found_paths_removable_np = np.concatenate([np.array(i) for i in found_paths_removable])

    nodes_on_paths, total_appearance = np.unique(found_paths_removable_np, return_counts=True)

    #for i in setP:
        #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
        #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))

    already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
    remaining_blocking_capacity -= 1


In [107]:
blocked_nodes = np.zeros(n)
#blocked_nodes[nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]-1]=1


In [108]:
blocked_nodes_df=pd.DataFrame(blocked_nodes).T
blocked_nodes_df.columns=(range(1,(n+1)))
blocked_nodes_df
blocked_nodes_df[already_blocked_nodes]=1

In [109]:
blocked_nodes_df.to_csv('blocked_nodes_df-n'+str(n)+'-m'+str(m)+'.csv',index=False)
